In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ['OPENAI_API_KEY']

In [3]:
from openai import OpenAI

client = OpenAI()

respose = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', "content": 'You are a helpful assistant'},
        {'role': 'user', 'content': 'こんにちは、私はmasaです。'}
    ],
    # logprobs=True
)

print(respose)

ChatCompletion(id='chatcmpl-BECuSdrsiAbnJqzVHVq4N3YizMsrR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='こんにちは、masaさん！お話できてうれしいです。今日はどんなことをお話ししましょうか？', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1742725916, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_b8bc95a0ac', usage=CompletionUsage(completion_tokens=27, prompt_tokens=22, total_tokens=49, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [23]:
print(respose.to_json(indent=2))

{
  "id": "chatcmpl-AWdZqGisaWQzvN6rdf3WBXlQKXMyc",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、masaさん！今日はどんなことをお手伝いしましょうか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1732341994,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0705bf87c0",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 22,
    "total_tokens": 40,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [19]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', "content": 'You are a helpful assistant'},
        {'role': 'user', 'content': 'こんにちは、私はmasaです。'}
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end='', flush=True)

こんにちは、masaさん！お元気ですか？何かお手伝いできることがあれば教えてください。

## JSON

In [24]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {'role': 'system', "content": '人物一覧を次のJSON形式で出力してください。\n{"people": ["aaa", "bbb"]}'},
        {'role': 'user', 'content': '昔あるところにお祖父さんとお婆さんがいました'}
    ],
    response_format={'type': 'json_object'}
    
)

print(response.choices[0].message.content)

{"people": ["お祖父さん", "お婆さん"]}


## 画像取り込み

In [25]:
image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "画像を説明してください。"},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)

この画像は本の表紙です。タイトルは「ChatGPT/LangChainによるチャットシステム構築[実践]入門」となっており、著者は吉田真吾氏と大嶋秀樹氏です。表紙にはカラフルなオウムのイラストが描かれており、背景は明るい青色です。また、「ChatGPT」という言葉が大きく表示されています。全体的に専門的な内容を示すデザインです。


# 2.6 Function Calling

In [3]:
import json

# Fanction callingの関数
def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [4]:
# LLMが使用できる関数一覧
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA, described in Engligh, but not country",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [5]:
messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-BQxslaGQJKkhqSoACfqYmEoJsqtf0",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_f24A6fWh3WPUmEasmkHGT5z8",
            "function": {
              "arguments": "{\"location\":\"Tokyo\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ],
        "annotations": []
      }
    }
  ],
  "created": 1745766415,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_f5bdcc3276",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 90,
    "total_tokens": 106,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
 

In [6]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

In [11]:
response_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_f24A6fWh3WPUmEasmkHGT5z8', function=Function(arguments='{"location":"Tokyo"}', name='get_current_weather'), type='function')], annotations=[])

In [7]:
response_message.to_dict()

{'content': None,
 'refusal': None,
 'role': 'assistant',
 'tool_calls': [{'id': 'call_f24A6fWh3WPUmEasmkHGT5z8',
   'function': {'arguments': '{"location":"Tokyo"}',
    'name': 'get_current_weather'},
   'type': 'function'}],
 'annotations': []}

In [8]:
messages

[{'role': 'user', 'content': '東京の天気はどうですか？'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_f24A6fWh3WPUmEasmkHGT5z8',
    'function': {'arguments': '{"location":"Tokyo"}',
     'name': 'get_current_weather'},
    'type': 'function'}],
  'annotations': []}]

In [10]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 使いたい関数は複数あるかもしれないのでループ
for tool_call in response_message.tool_calls:
    # 関数を実行
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # 関数の実行結果を会話履歴としてmessagesに追加
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

{"location": "Tokyo", "temperature": "10", "unit": null}


In [48]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "東京の天気はどうですか？"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_wBBKUzh5FAZXKApZeA9GyMwS",
        "function": {
          "arguments": "{\"location\":\"Tokyo\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_wBBKUzh5FAZXKApZeA9GyMwS",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Tokyo\", \"temperature\": \"10\", \"unit\": null}"
  }
]


In [49]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(second_response.to_json(indent=2))

{
  "id": "chatcmpl-AWfJ6iVliuU5PxPwxHFe2rjFR0Meb",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "現在の東京の気温は10度です。他にも知りたいことがあれば教えてくださいね。",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1732348644,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "system_fingerprint": "fp_831e067d82",
  "usage": {
    "completion_tokens": 24,
    "prompt_tokens": 57,
    "total_tokens": 81,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


## プロンプトエンジアリング

In [50]:
prompt = '''\
以下の料理のレシピを考えてください。

料理名: """
{dish}
"""
'''


def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt.format(dish=dish)},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("カレー")
print(recipe)

カレーの基本的なレシピをご紹介します。お好みで具材を変更したり、スパイスの量を調整したりして、自分だけのカレーを楽しんでください。

### 材料（4人分）

- 鶏肉または牛肉または豚肉: 400g（お好きな部位で）
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約180g）
- 水: 800ml
- サラダ油: 大さじ2
- 塩: 適量
- こしょう: 適量
- Optional: ガーリックパウダー、しょうがパウダー、クミン、ターメリック、チリパウダー（お好みで）

### 作り方

1. **下準備**:
   - 玉ねぎを薄切りにする。
   - にんじんとじゃがいもは皮をむいて、一口大に切る。
   - 肉は一口大に切り、塩とこしょうを振って下味をつける。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、薄切りにした玉ねぎを中火で炒める。
   - 玉ねぎが透明になるまで炒め、その後に肉を加えて表面が白くなるまで炒める。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜる。
   - さらに約2〜3分炒める。

4. **煮込む**:
   - 水を加え、沸騰したらアクを取り除く。
   - 蓋をして中弱火で約15〜20分間、野菜が柔らかくなるまで煮る。

5. **カレールーを溶かす**:
   - 火を止めてカレールーを加え、よく混ぜる。再び弱火にし、ルーが完全に溶けるまで煮込む。

6. **味を調える**:
   - お好みに応じて、ガーリックパウダーやしょうがパウダーなどのスパイスを加える。味見をして、必要なら塩で調整する。

7. **完成**:
   - 煮込んだカレーを器に盛り、ご飯やナンと一緒にお召し上がりください。

### お好みのトッピング
- 刻みパセリやコリアンダー
- 笹かまぼこやゆで卵
- チーズやサワークリーム

このレシピは基本のカレーですが、具材やスパイス類を変えることで、お好みのカレーを楽しんでください！おいしく仕上げてくださいね。


In [52]:
prompt

'以下の料理のレシピを考えてください。\n\n料理名: """\n{dish}\n"""\n'